In [ ]:
!pip install ragas

In [ ]:
import pandas as pd

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "use_your_key"

In [ ]:
df = pd.read_excel("/content/only_rag_evaluation (1).xlsx")

In [ ]:
from ragas.metrics import (
    context_relevancy,
    answer_relevancy,
    faithfulness,
    context_recall,
)
from ragas.metrics.critique import harmfulness
from ragas import evaluate

In [ ]:
df.columns

Index(['Paper', 'Question', 'Original_Answer', 'Model_Answer', 'Context'], dtype='object')

In [ ]:
df

,Paper,Question,Original_Answer,Model_Answer,Context
0,Evaluating the Performance of a Speech Recogni...,How can the Levenshtein distance be used to id...,Calculate the Levenshtein distance between all...,"Great, let's dive into the details of the pape...",Regarding the paper 'Evaluating the Performanc...
1,Detecting and Extracting Events from Text Docu...,In the paper Detecting and Extracting Events f...,The BioNLP 2009 Shared Task BIBREF195 was base...,"In the paper ""Detecting and Extracting Events...",Regarding the paper 'Detecting and Extracting ...
2,Detecting and Extracting Events from Text Docu...,What are the three classes of events according...,"Based on the paper [Kenny 2003], Aristotle's t...",The authors also mention that they use a combi...,Regarding the paper 'Detecting and Extracting ...
3,Detecting and Extracting Events from Text Docu...,What is the difference between an activity and...,"Based on the paper ""The Differences Between an...","Based on the paper ""The Differences Between an...",Regarding the paper 'Detecting and Extracting ...
4,The Grail theorem prover: Type theory for synt...,In the paper The Grail theorem prover: Type th...,This chapter describes the underlying formalis...,"In the paper ""The Grail theorem prover: Type ...",Regarding the paper 'The Grail theorem prover:...
5,The Grail theorem prover: Type theory for synt...,What are the three main components of a type-l...,"Logic, syntax-semantics interface, and lexicon.",The three main components of a type-logical gr...,Regarding the paper 'The Grail theorem prover:...
6,The Grail theorem prover: Type theory for synt...,Who is the author of the paper?,"Richard Moot is the author of the paper ""The G...",The authors of the papers you mentioned are:Ho...,Regarding the paper 'The Grail theorem prover:...


In [ ]:
# Define a dictionary with current column names as keys and new names as values
column_name_mapping = {
    'Question': 'question',
    'Original_Answer': 'ground_truths',
    'Model_Answer': 'answer',
    'Context': 'contexts',
    # Add more as needed
}
# Use the rename method to change column names
df.rename(columns=column_name_mapping, inplace=True)
# Print the DataFrame to verify the changes
print(df)

                                               Paper  \
0  Evaluating the Performance of a Speech Recogni...   
1  Detecting and Extracting Events from Text Docu...   
2  Detecting and Extracting Events from Text Docu...   
3  Detecting and Extracting Events from Text Docu...   
4  The Grail theorem prover: Type theory for synt...   
5  The Grail theorem prover: Type theory for synt...   
6  The Grail theorem prover: Type theory for synt...   

                                            question  \
0  How can the Levenshtein distance be used to id...   
1  In the paper Detecting and Extracting Events f...   
2  What are the three classes of events according...   
3  What is the difference between an activity and...   
4  In the paper The Grail theorem prover: Type th...   
5  What are the three main components of a type-l...   
6                    Who is the author of the paper?   

                                       ground_truths  \
0  Calculate the Levenshtein distance between 

In [ ]:
# Replace 'column_to_drop' with the actual name of the column you want to drop
column_to_drop = 'Paper'
# Use the drop method to remove the specified column
df.drop(column_to_drop, axis=1, inplace=True)
# Print the DataFrame to verify the changes
print(df)

                                            question  \
0  How can the Levenshtein distance be used to id...   
1  In the paper Detecting and Extracting Events f...   
2  What are the three classes of events according...   
3  What is the difference between an activity and...   
4  In the paper The Grail theorem prover: Type th...   
5  What are the three main components of a type-l...   
6                    Who is the author of the paper?   

                                       ground_truths  \
0  Calculate the Levenshtein distance between all...   
1  The BioNLP 2009 Shared Task BIBREF195 was base...   
2  Based on the paper [Kenny 2003], Aristotle's t...   
3  Based on the paper "The Differences Between an...   
4  This chapter describes the underlying formalis...   
5    Logic, syntax-semantics interface, and lexicon.   
6  Richard Moot is the author of the paper "The G...   

                                              answer  \
0  Great, let's dive into the details of the p

In [ ]:
from datasets import Dataset
# Assuming df is your DataFrame
df['contexts'] = df['contexts'].apply(lambda x: [str(x)]).tolist()
df['ground_truths'] = df['ground_truths'].apply(lambda x: [str(x)]).tolist()
data_dict = {
    'question': df['question'].tolist(),
    'ground_truths': df['ground_truths'].tolist(),
    'answer': df['answer'].tolist(),
    'contexts': df['contexts'],
}
# Convert the 'contexts' column to a list of strings
custom_dataset = Dataset.from_dict(data_dict)

# Now, you can use custom_dataset in the evaluate function
result = evaluate(
    custom_dataset,
    metrics=[
        context_relevancy,
        faithfulness,
        answer_relevancy,
        context_recall,
        harmfulness,
    ],
)
result

evaluating with [context_relevancy]


100%|██████████| 1/1 [00:27<00:00, 27.55s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [01:10<00:00, 70.32s/it]


evaluating with [answer_relevancy]


...

evaluating with [context_recall]


100%|██████████| 1/1 [00:41<00:00, 41.60s/it]


evaluating with [harmfulness]


100%|██████████| 1/1 [00:14<00:00, 14.06s/it]


{'context_relevancy': 0.4635, 'faithfulness': 0.2500, 'answer_relevancy': 0.9614, 'context_recall': 0.5000, 'harmfulness': 0.0000}